# **Import Modules & Libraries**

In [1]:
#!pip install pytorch-ignite

In [2]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import Engine, Events
from ignite.utils import setup_logger

# **Model Definition**

In [3]:
class MLP(nn.Module):
  def __init__(self, input_dim, N1, N2, output_dim):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(input_dim,N1)
    self.fc2 = nn.Linear(N1, N2)
    self.fc3 = nn.Linear(N2, output_dim)
    self.dropout = nn.Dropout(0.2)
    self.relu = nn.ReLU()
  
  def forward(self, x):
    out=self.dropout(self.relu(self.fc1(x)))
    out=self.dropout(self.relu(self.fc2(out)))
    out=self.fc3(out)        
    return out

# **Data Preparation**

In [4]:
def prepare_data(batch_size):
  # transform to normalize the data
  transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
  
  # Download and load the training and test datasets
  trainset = MNIST(root='MNIST', download=True, train=True, transform=transform)
  testset = MNIST(root='MNIST', download=True, train=False, transform=transform)

  trainset_size = round(len(trainset) * 0.8)
  validset_size = len(trainset) - trainset_size
  trainset, validset = random_split(trainset, [trainset_size, validset_size])

  train_dl = DataLoader(trainset, batch_size=batch_size, shuffle=True)
  valid_dl = DataLoader(validset, batch_size=batch_size, shuffle=True)
  test_dl = DataLoader(testset, batch_size=batch_size, shuffle=False)

  return train_dl, valid_dl, test_dl

# **Setting**

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP(784, 128, 64, 10)
model.to(device)  # Move model before creating optimizer
print(model)
optimizer = Adam(model.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()

MLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
)


# **Training**

In [6]:
def train_step(engine, batch):
  model.train()
  optimizer.zero_grad()
  x, y = batch[0].to(device), batch[1].to(device)
  y_pred = model(x.view(x.shape[0], -1))
  loss = criterion(y_pred, y)
  loss.backward()
  optimizer.step()
  return loss.item()

def validation_step(engine, batch):
  model.eval()
  with torch.no_grad():
    inputs = batch[0].to(device)
    labels = batch[1].to(device)
    outputs = model(inputs.view(inputs.shape[0], -1))
    _,predicted = torch.max(outputs.data, 1)      
    accuracy = (predicted == labels).sum().item()/labels.size(0)
  return accuracy

In [7]:
def run(model, device, optimizer, criterion, epochs, train_dl, valid_dl):

  # Trainer
  trainer = Engine(train_step)
  trainer.logger = setup_logger("trainer")

  # Evaluator
  Evaluator = Engine(validation_step)
  Evaluator.logger = setup_logger("Evaluator")
  
  @trainer.on(Events.ITERATION_COMPLETED(every=100))
  def log_training_loss(engine):
    print(f"ITERATION - Loss: {engine.state.output:.2f}")
        
  @trainer.on(Events.EPOCH_COMPLETED)
  def log_training_results(engine):
    Evaluator.run(train_dl)
    print(f"Training Results - Epoch: {trainer.state.epoch} Accuracy: {Evaluator.state.output:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED)
  def log_validation_results(engine):
    Evaluator.run(valid_dl)
    print(f"Validation Results - Epoch: {trainer.state.epoch} Accuracy: {Evaluator.state.output:.2f}")

  @trainer.on(Events.EPOCH_COMPLETED | Events.COMPLETED)
  def log_time(engine):
    print(f"{trainer.last_event_name.name} took { trainer.state.times[trainer.last_event_name.name]} seconds")

  trainer.run(train_dl, max_epochs=epochs)

In [8]:
if __name__ == "__main__":
  # Data Preparation
  train_dl, valid_dl, test_dl = prepare_data(batch_size=64)

  run(model, device, optimizer, criterion, 10, train_dl, valid_dl)

2022-11-11 02:00:59,390 trainer INFO: Engine run starting with max_epochs=10.


ITERATION - Loss: 0.46
ITERATION - Loss: 0.38
ITERATION - Loss: 0.61
ITERATION - Loss: 0.29
ITERATION - Loss: 0.44
ITERATION - Loss: 0.44
ITERATION - Loss: 0.51


2022-11-11 02:01:11,727 Evaluator INFO: Engine run starting with max_epochs=1.
2022-11-11 02:01:19,327 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:07.599
2022-11-11 02:01:19,329 Evaluator INFO: Engine run complete. Time taken: 00:00:07.601
2022-11-11 02:01:19,332 Evaluator INFO: Engine run starting with max_epochs=1.


Training Results - Epoch: 1 Accuracy: 0.86


2022-11-11 02:01:21,264 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:01.928
2022-11-11 02:01:21,266 Evaluator INFO: Engine run complete. Time taken: 00:00:01.929
2022-11-11 02:01:21,269 trainer INFO: Epoch[1] Complete. Time taken: 00:00:21.874


...

ITERATION - Loss: 0.08


2022-11-11 02:04:07,127 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:07.743
2022-11-11 02:04:07,129 Evaluator INFO: Engine run complete. Time taken: 00:00:07.745
2022-11-11 02:04:07,134 Evaluator INFO: Engine run starting with max_epochs=1.


Training Results - Epoch: 10 Accuracy: 1.00


2022-11-11 02:04:09,022 Evaluator INFO: Epoch[1] Complete. Time taken: 00:00:01.885
2022-11-11 02:04:09,023 Evaluator INFO: Engine run complete. Time taken: 00:00:01.887
2022-11-11 02:04:09,028 trainer INFO: Epoch[10] Complete. Time taken: 00:00:18.501
2022-11-11 02:04:09,033 trainer INFO: Engine run complete. Time taken: 00:03:09.642


Validation Results - Epoch: 10 Accuracy: 1.00
EPOCH_COMPLETED took 8.852379322052002 seconds
COMPLETED took 189.64156651496887 seconds
